<a href="https://colab.research.google.com/github/maxzameer/Brainstorm/blob/main/deep%20learning/cat_vs_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [29]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset


microsoft-catsvsdogs-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [30]:
import zipfile

In [31]:
zip_ref = zipfile.ZipFile('/content/microsoft-catsvsdogs-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [32]:
! pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import splitfolders
splitfolders.ratio('/content/PetImages', output="/content/data", seed=1337, ratio=(0.8,0.2 ))

Copying files: 25002 files [00:06, 3939.76 files/s]


In [50]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [61]:

train_df = keras.utils.image_dataset_from_directory(
    directory = '/content/data/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,    
    image_size = (256,256)
)

val_df = keras.utils.image_dataset_from_directory(
    directory = '/content/data/val',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (256,256)
)

Found 19998 files belonging to 2 classes.
Found 5002 files belonging to 2 classes.


In [36]:
train_df

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [55]:
def normalize(image,label):
  image = tf.cast(image/255,tf.float32)
  return image,label

In [56]:
train_df = train_df.map(normalize)
val_df = val_df.map(normalize)

In [57]:

# create CNN model

model = Sequential()
model.add(Conv2D(32,kernel_size = (3,3),padding = 'valid',activation ='relu',input_shape =(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2 , padding = 'valid'))

model.add(Conv2D(32,kernel_size = (3,3),padding = 'valid',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2 , padding = 'valid'))

model.add(Conv2D(32,kernel_size = (3,3),padding = 'valid',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2 , padding = 'valid'))

model.add(Flatten())
model.add(Dense(128,activation ='relu'))
model.add(Dense(64,activation ='relu'))
model.add(Dense(1,activation ='sigmoid'))



In [58]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 30, 30, 32)      

In [59]:
model.compile(optimizer='adam',loss ='binary_crossentropy',metrics = ['accuracy'])


In [64]:
history = model.fit(train_df,epochs=10,validation_data=val_df)


Epoch 1/10
 18/625 [..............................] - ETA: 46s - loss: 0.5102 - accuracy: 0.7604

InvalidArgumentError: ignored